In [52]:
import os
from openai import OpenAI
from evaluate import load
from transformers import WhisperProcessor
from evaluate import load

wer = load("wer")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")

In [103]:
client = OpenAI(
  organization='org-cQcini8CmgApuWQd7TW63AbZ',
  project='proj_l5hBsnWNcaXmXjNkaNabPmDw',
  api_key = 'sk-proj-Jzb7ixiKOmmUDVdYYenIT3BlbkFJyZA4dzrp7BFVO4fU1dvg'
)


audio_file= open("/Users/raghavgarg/Desktop/CS224S/Data/Audio Recordings/CAR0001.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file,
  response_format="verbose_json",
  timestamp_granularities=["word"]
)
print(transcription)

Transcription(text="what brought you in today? Sure, I'm just having a lot of chest pain and so I thought I should get it checked out. Okay, and before we start could you remind me of your gender and age? Sure, I'm 39. I'm a male. Okay, and so when did this chest pain start? It started last night but it's becoming sharper. Okay, and where is this pain located? It's located on the left side of my chest. Okay, and so how long has it been going on for than if it started last night? So I guess it would be a couple hours now, maybe like eight. Okay, has it been constant throughout that time or changing? I would say it's been pretty constant, yeah. Okay, and how would you describe the pain? People will use words sometimes like sharp, burning, achy. I'd say it's pretty sharp, yeah. Sharp, okay. Anything that you have done, tried since last night that's made the pain better? Not laying down helps. Okay, so do you find laying down makes the pain worse? Yes, definitely. Okay, do you find that th

In [104]:
system_prompt = '''You are a helpful assistant for this speaker diarisation task. Your task is to indicate when the Doctor is speaking with D and the patient speaking with P. Only add necessary punctuation such as periods, commas, and capitalization, and use only the context provided. For example: D: How may I help you?

P: Hi, yes it's nice to meet you. I've been having this cough that's been going on for the last few days and I have had difficulty breathing.

D: Oh, I'm so sorry to hear that. When did this start?

P: Oh, so the cough started about'''

def generate_corrected_transcript(system_prompt, transcription_obj):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": transcription_obj.text
            }
        ]
    )
    return response

response = generate_corrected_transcript(0, system_prompt, transcription)

In [111]:
def compute_wer(batch):
    def parse_transcript(transcript):
        speaker_segments, d_speaker_segments = [], []
        lines = transcript.split('\n')

        for line in lines:
            if ':' in line:
                speaker, text = line.split(':', 1)
                text = processor.tokenizer.normalize(text)
                words = text.strip().split()
                d_speaker_segments += [f'{speaker}_{word}' for word in words]
                speaker_segments += [word for word in words]
        return ' '.join(speaker_segments), ' '.join(d_speaker_segments)

    batch["pred_norm"], batch["pred_norm_d"] = parse_transcript(batch["pred"])
    batch["ref_norm"], batch["ref_norm_d"] = parse_transcript(batch["ref"])
    batch["wer_d"] = wer.compute(predictions=[batch["pred_norm_d"]], references=[batch["ref_norm_d"]])
    batch["wer"] = wer.compute(predictions=[batch["pred_norm"]], references=[batch["ref_norm"]])
    return batch

candidate_transcript = response.choices[0].message.content
with open("/Users/raghavgarg/Desktop/CS224S/Data/Clean Transcripts/CAR0001.txt", 'r') as f:
    reference_transcript = f.read()

example = [{"ref": reference_transcript, "pred": candidate_transcript}]
result = list(map(compute_wer, example))

In [112]:
result

[{'ref': "D: What brought you in today?\n\nP: Sure, I'm I'm just having a lot of chest pain and and so I thought I should get it checked out.\n\nD: OK, before we start, could you remind me of your gender and age? \n\nP: Sure 39, I'm a male.\n\nD: OK, and so when did this chest pain start?\n\nP: It started last night, but it's becoming sharper.\n\nD: OK, and where is this pain located? \n\nP: It's located on the left side of my chest.\n\nD: OK, and, so how long has it been going on for then if it started last night?\n\nP: So I guess it would be a couple of hours now, maybe like 8.\n\nD: OK. Has it been constant throughout that time, or uh, or changing? \n\nP: I would say it's been pretty constant, yeah.\n\nD: OK, and how would you describe the pain? People will use words sometimes like sharp, burning, achy. \n\nP: I'd say it's pretty sharp, yeah.\n\nD: Sharp OK. Uh, anything that you have done tried since last night that's made the pain better?\n\nP: Um not laying down helps.\n\nD: OK, 

In [47]:
import numpy as np
import string
from difflib import SequenceMatcher

def calculate_word_der(candidate, reference):
    def parse_transcript(transcript):
        speaker_segments = []
        lines = transcript.split('\n')
        translator = str.maketrans('', '', string.punctuation)

        for line in lines:
            if ':' in line:
                speaker, text = line.split(':', 1)
                text = text.translate(translator)
                words = text.strip().split()
                speaker_segments.append((speaker.strip(), words))

        return speaker_segments

    candidate_segments = parse_transcript(candidate)
    reference_segments = parse_transcript(reference)

    candidate_words = [(speaker, word) for speaker, words in candidate_segments for word in words]
    reference_words = [(speaker, word) for speaker, words in reference_segments for word in words]

    just_candidate_words = [word for _, words in candidate_segments for word in words]
    just_reference_words = [word for _, words in reference_segments for word in words]

    matcher = SequenceMatcher(None, just_candidate_words, just_reference_words)
    matches = matcher.get_matching_blocks()

    false_alarms = 0
    missed_detections = 0
    speaker_confusions = 0

    candidate_index = 0
    reference_index = 0

    for match in matches:
        print("new match", match.a, candidate_index, match.b, reference_index, match.size)
        while candidate_index < match.a:
            print(just_candidate_words[max(candidate_index-5, 0): min(candidate_index+5, len(candidate_words))])
            print(just_reference_words[max(reference_index-5, 0): min(reference_index+5, len(reference_words))])
            false_alarms += 1
            candidate_index += 1

        while reference_index < match.b:
            print(just_candidate_words[max(candidate_index-5, 0): min(candidate_index+5, len(candidate_words))])
            print(just_reference_words[max(reference_index-5, 0): min(reference_index+5, len(reference_words))])
            missed_detections += 1
            reference_index += 1

        for i in range(match.size):
            candidate_speaker, candidate_word = candidate_words[candidate_index]
            reference_speaker, reference_word = reference_words[reference_index]

            if candidate_speaker != reference_speaker:
                speaker_confusions += 1
                print("speaker confusion", candidate_speaker, reference_speaker, candidate_word, reference_word)

            candidate_index += 1
            reference_index += 1

    while candidate_index < len(candidate_words):
        false_alarms += 1
        candidate_index += 1

    while reference_index < len(reference_words):
        missed_detections += 1
        reference_index += 1

    total_reference_words = len(reference_words)
    der = (false_alarms + missed_detections + speaker_confusions) / total_reference_words

    return der

print(f'Diarization Error Rate (DER): {der}')


new match 0 0 0 0 7
new match 7 7 8 7 7
['you', 'in', 'today', 'Sure', 'Im', 'just', 'having', 'a', 'lot', 'of']
['you', 'in', 'today', 'Sure', 'Im', 'Im', 'just', 'having', 'a', 'lot']
new match 14 14 16 15 10
['a', 'lot', 'of', 'chest', 'pain', 'and', 'so', 'I', 'thought', 'I']
['a', 'lot', 'of', 'chest', 'pain', 'and', 'and', 'so', 'I', 'thought']
new match 26 24 27 26 13
['should', 'get', 'it', 'checked', 'out', 'Okay', 'and', 'before', 'we', 'start']
['should', 'get', 'it', 'checked', 'out', 'OK', 'before', 'we', 'start', 'could']
['get', 'it', 'checked', 'out', 'Okay', 'and', 'before', 'we', 'start', 'could']
['should', 'get', 'it', 'checked', 'out', 'OK', 'before', 'we', 'start', 'could']
['it', 'checked', 'out', 'Okay', 'and', 'before', 'we', 'start', 'could', 'you']
['should', 'get', 'it', 'checked', 'out', 'OK', 'before', 'we', 'start', 'could']
new match 40 39 40 40 4
['your', 'gender', 'and', 'age', 'Sure', 'Im', '39', 'Im', 'a', 'male']
['your', 'gender', 'and', 'age', 'Su